In [1]:
import os

In [2]:
%pwd

'c:\\Users\\amitt\\Documents\\ML-OPS\\text-summarization\\reseach'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\amitt\\Documents\\ML-OPS\\text-summarization'

In [5]:
!nvidia-smi

Sat Feb 15 17:48:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 561.19                 Driver Version: 561.19         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   43C    P0             14W /   86W |       0MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
!pip install transformers[sentencepiece] 

!pip install sacrebleu

!pip install py7zr 
!pip install -q

You should consider upgrading via the 'C:\Users\amitt\Documents\ML-OPS\text-summarization\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\amitt\Documents\ML-OPS\text-summarization\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\amitt\Documents\ML-OPS\text-summarization\myenv\Scripts\python.exe -m pip install --upgrade pip' command.
ERROR: You must give at least one requirement to install (see "pip help install")
You should consider upgrading via the 'C:\Users\amitt\Documents\ML-OPS\text-summarization\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [7]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

You should consider upgrading via the 'C:\Users\amitt\Documents\ML-OPS\text-summarization\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


Found existing installation: transformers 4.48.3
Uninstalling transformers-4.48.3:
  Successfully uninstalled transformers-4.48.3
Found existing installation: accelerate 1.3.0
Uninstalling accelerate-1.3.0:
  Successfully uninstalled accelerate-1.3.0
  Using cached transformers-4.48.3-py3-none-any.whl (9.7 MB)
  Using cached accelerate-1.3.0-py3-none-any.whl (336 kB)


You should consider upgrading via the 'C:\Users\amitt\Documents\ML-OPS\text-summarization\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:

    root_dir : Path
    data_path : Path
    model_name : Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int



In [17]:
from src.textsummarizer.constant import *
from src.textsummarizer.utils.common import read_yaml, create_directories

In [18]:
#  this goes into the configuration file under config folder
class ConfigurationManager:
    def __init__( self,config_filepath = CONFIG_PATH, params_filepath = PARAMS_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:

            config = self.config.model_trainer
            print(config)
            params = self.params.TrainingArguments

            # creating root directory for config.root_dir

            create_directories([config.root_dir])

            model_trainer_config = ModelTrainerConfig(

                root_dir = config.root_dir,
                data_path = config.data_path,
                model_name = config.model_name,
                num_train_epochs = params.num_train_epochs,
                warmup_steps = params.warmup_steps,
                per_device_train_batch_size = params.per_device_train_batch_size,
                per_device_eval_batch_size = params.per_device_eval_batch_size,
                weight_decay = params.weight_decay,
                logging_steps = params.logging_steps,
                evaluation_strategy = params.evaluation_strategy,
                eval_steps = params.eval_steps,
                save_steps = params.save_steps,
                gradient_accumulation_steps =  params.gradient_accumulation_steps
            )

            return model_trainer_config


In [19]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

In [ ]:
class ModelTrainer:

    def __init__(self, config: ModelTrainerConfig):

        self.config = config

    def train(self):

        device = "cuda" if torch.cuda.is_available() else "cpu"

        tokenizer = AutoTokenizer.from_pretrained(self.config.model_name) # load tokenizr
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_name).to(device) #

        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model=model)

        # loading the data

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(

            output_dir = self.config.root_dir, 
            num_train_epochs = self.config.num_train_epochs, 
            warmup_steps = self.config.warmup_steps,
            per_device_train_batch_size = self.config.per_device_train_batch_size, 
            per_device_eval_batch_size = self.config.per_device_eval_batch_size,
            weight_decay = self.config.weight_decay, 
            logging_steps = self.config.logging_steps, 
            evaluation_strategy = self.config.evaluation_strategy, 
            eval_steps = self.config.eval_steps, 
            save_steps = 1e6,
            gradient_accumulation_steps = self.config.gradient_accumulation_steps,
            # remove_unused_columns=False
        )

        trainer = Trainer(model=model, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])
        

        trainer.train()

        model.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-trained-model"))  

        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))



In [21]:
config = ConfigurationManager()

model_trainer_config = config.get_model_trainer_config()

model_trainer = ModelTrainer(config=model_trainer_config)

model_trainer.train()

[2025-02-15 18:00:39,931: INFO: common: yaml file: config\config.yaml loaded successfully]


[2025-02-15 18:00:39,932: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-15 18:00:39,934: INFO: common: created directory at: artifacts]
{'root_dir': 'artifacts/model_training', 'data_path': 'artifacts/data_transformation/samsum_dataset', 'model_name': 'google/pegasus-cnn_dailymail'}
[2025-02-15 18:00:39,934: INFO: common: created directory at: artifacts/model_training]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\amitt\AppData\Local\Temp\ipykernel_20752\2232375563.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=trainer_args,


Step,Training Loss,Validation Loss


c:\Users\amitt\Documents\ML-OPS\text-summarization\myenv\lib\site-packages\transformers\modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
